In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
from pathlib import Path
import pandas as pd
from tqdm import tqdm
import pickle
import numpy as np

In [3]:
from openslide import OpenSlide

In [4]:
from exact_sync.v1.api.annotations_api import AnnotationsApi
from exact_sync.v1.api.images_api import ImagesApi
from exact_sync.v1.api.image_sets_api import ImageSetsApi
from exact_sync.v1.api.annotation_types_api import AnnotationTypesApi
from exact_sync.v1.api.products_api import ProductsApi

from exact_sync.v1.models import ImageSet, Product, AnnotationType, Image, Annotation
from exact_sync.v1.rest import ApiException
from exact_sync.v1.configuration import Configuration
from exact_sync.v1.api_client import ApiClient

In [5]:
import sys
sys.path.append("../..")
from registration_tree import Rect, QuadTree

## Find and load slides from HDD

In [6]:
slide_list = {path.name: path for path in Path(r"D:\Datasets\ScannerStudy").glob("*/*/*.*") 
                      if path.suffix in [".svs", ".ndpi", ".tif"]}

slide_list

{'A_CCMCT_183715A_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_183715A_1.svs'),
 'A_CCMCT_22108_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_22108_1.svs'),
 'A_CCMCT_29609B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_29609B_1.svs'),
 'A_CCMCT_380609B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_380609B_1.svs'),
 'A_CCMCT_518711B_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/CCMCT/A_CCMCT_518711B_1.svs'),
 'A_BB_563476_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_563476_1.svs'),
 'A_BB_563479_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_563479_1.svs'),
 'A_BB_568320_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_568320_1.svs'),
 'A_BB_568381_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_568381_1.svs'),
 'A_BB_574162_1.svs': WindowsPath('D:/Datasets/ScannerStudy/Aperio/Cyto/A_BB_574162_1.svs'),
 'Z_CCMCT_183715A_1.tif': Windo

## Connect to EXACT

In [7]:
configuration = Configuration()
configuration.username = 'marzahl' #'exact'
configuration.password = '####' #'exact'
configuration.host = "https://exact.cs.fau.de" #"http://127.0.0.1:8000"

client = ApiClient(configuration)

image_sets_api = ImageSetsApi(client)
annotations_api = AnnotationsApi(client)
annotation_types_api = AnnotationTypesApi(client)
images_api = ImagesApi(client)
product_api = ProductsApi(client)

## Download RefPoints annotation

In [8]:
annotation_types_ref_rect = None

for product in product_api.list_products(name="References").results:
    for annotation_type in annotation_types_api.list_annotation_types(product=product.id, name="BoundingBox").results:
        annotation_types_ref_rect = annotation_type
        
annotation_types_ref_rect

{'area_hit_test': False,
 'closed': True,
 'color_code': '#FF0000',
 'default_height': 5000,
 'default_width': 5000,
 'enable_blurred': False,
 'enable_concealed': False,
 'id': 412,
 'name': 'BoundingBox',
 'node_count': 0,
 'product': 105,
 'sort_order': 0,
 'vector_type': 1}

In [9]:
annotations = []

image_set = image_sets_api.list_image_sets(name="CCMCT_Aperio", expand="product_set,product_set.annotationtype_set").results[0]

for image_id in image_set.images:
    
    image = images_api.retrieve_image(id=image_id)
    
    for anno in annotations_api.list_annotations(image=image_id,
                                                 deleted=False, fields="annotation_type,id,image,vector,unique_identifier").results:
        
        if anno.annotation_type != annotation_types_ref_rect.id:
            annotations.append([image.id, image.name, Path(image.name).stem.split("_")[2].split("_")[0], anno.id, anno.vector, anno.unique_identifier, anno.annotation_type])
        
annotations = pd.DataFrame(annotations, columns=["image_id", "image_name", "image_name_short", "id", "vector", "unique_identifier", "annotation_type"])
annotations

,image_id,image_name,image_name_short,id,vector,unique_identifier,annotation_type
0,10416,A_CCMCT_518711B_1.svs,518711B,2755281,"{'x1': 13431, 'x2': 13481, 'y1': 8432, 'y2': 8...",ca3fde15-30ed-45bd-bd58-00aadf29eba9,384
1,10416,A_CCMCT_518711B_1.svs,518711B,2755282,"{'x1': 15560, 'x2': 15610, 'y1': 25727, 'y2': ...",0acc9849-e7aa-4657-8055-bb7e8b4a31e0,385
2,10416,A_CCMCT_518711B_1.svs,518711B,2755283,"{'x1': 13378, 'x2': 13428, 'y1': 42978, 'y2': ...",e44ca58f-3316-4d02-b6b1-1d44f2664682,386
3,10416,A_CCMCT_518711B_1.svs,518711B,2755284,"{'x1': 12126, 'x2': 12176, 'y1': 58187, 'y2': ...",99a54973-1bf3-4cbc-a55b-a09b62e0e38f,387
4,10416,A_CCMCT_518711B_1.svs,518711B,2755285,"{'x1': 13405, 'x2': 13455, 'y1': 71704, 'y2': ...",ecc654b9-910b-4792-955b-52070dbff958,388
...,...,...,...,...,...,...,...
120,10398,A_CCMCT_22108_1.svs,22108,2755401,"{'x1': 88422, 'x2': 88472, 'y1': 11835, 'y2': ...",25aa751c-06cf-463c-ace9-8397c567972b,407
121,10398,A_CCMCT_22108_1.svs,22108,2755402,"{'x1': 88406, 'x2': 88456, 'y1': 28051, 'y2': ...",8a4bc90e-afd7-4a3a-8028-1a6c24ab97f8,408
122,10398,A_CCMCT_22108_1.svs,22108,2755403,"{'x1': 88353, 'x2': 88403, 'y1': 44115, 'y2': ...",edce5b71-592c-49f4-b70c-bf8f8faec9a0,409
123,10398,A_CCMCT_22108_1.svs,22108,2755404,"{'x1': 88463, 'x2': 88513, 'y1': 60497, 'y2': ...",cd11d8de-2fc8-48c1-9699-a4a86746c0fb,410


## Add some helper variables

In [10]:
annotations["x1"] = [vector['x1'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y1"] = [vector['y1'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["x2"] = [vector['x2'] if type(vector) is dict else 0 for vector in annotations["vector"]]
annotations["y2"] = [vector['y2'] if type(vector) is dict else 0 for vector in annotations["vector"]]

annotations["center_x"] = [x1 + ((x2-x1) / 2) for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["center_y"] = [y1 + ((y2-y1) / 2) for y1, y2 in zip(annotations["y1"], annotations["y2"])]

annotations["anno_width"] = [x2-x1 for x1, x2 in zip(annotations["x1"], annotations["x2"])]
annotations["anno_height"]= [y2-y1 for y1, y2 in zip(annotations["y1"], annotations["y2"])]

## Load QuadTree for each scanner and transform coordinates

In [11]:
transformed_annotations_to_upload = []

for image_set_name in tqdm(["CCMCT_2.0HT", "CCMCT_Axio", "CCMCT_S210"]):
    image_set = image_sets_api.list_image_sets(name=image_set_name, expand="product_set,product_set.annotationtype_set").results[0]

    for image_id in image_set.images:
        
        target_image = images_api.retrieve_image(id=image_id)
        
        source_annos = annotations[annotations["image_name_short"] == Path(target_image.name).stem.split("_")[2].split("_")[0]]
        
        source_name = list(source_annos["image_name"].unique())[0]
        target_name = target_image.name
        
        qtree = pickle.load(open(f'../../CCMCT/{Path(source_name).stem}-To-{Path(target_name).stem}.pickle', "rb" ))
        qtree.source_slide = OpenSlide(str(slide_list[source_name]))
        qtree.target_slide = OpenSlide(str(slide_list[target_name]))
        
        for id, source_anno in source_annos.iterrows():

            box = [source_anno.center_x, source_anno.center_y, source_anno.anno_width, source_anno.anno_height]

            trans_box = qtree.transform_boxes(np.array([box]))[0]

            new_x1 = int(trans_box[0] - trans_box[2] // 2)
            new_y1 = int(trans_box[1] - trans_box[3] // 2)
            new_x2 = int(trans_box[0] + trans_box[2] // 2)
            new_y2 = int(trans_box[1] + trans_box[3] // 2)

            vector = {'x1': new_x1, 'x2': new_x2, 'y1': new_y1, 'y2': new_y2}

            anno = Annotation(annotation_type=source_anno.annotation_type, 
                                  vector=vector, 
                                  image=target_image.id,
                                 unique_identifier=source_anno.unique_identifier)

            transformed_annotations_to_upload.append(anno)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:09<00:00,  3.04s/it]


In [12]:
transformed_annotations_to_upload[:3]

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10410,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': '95183f6e-0a00-4de4-9c97-fedfec655a27',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 20328, 'x2': 20382, 'y1': 16747, 'y2': 16801},
  'verified_by_user': None},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': None,
  'concealed': None,
  'deleted': None,
  'description': None,
  'id': None,
  'image': 10410,
  'last_edit_time': None,
  'last_editor': None,
  'meta_data': None,
  'time': None,
  'unique_identifier': 'cc3def2b-5631-4619-ab08-51f2a69c667d',
  'uploaded_media_files': [],
  'user': None,
  'vector': {'x1': 20265, 'x2': 20319, 'y1': 35440, 'y2': 35494},
  'verified_by_user': None},
 {'annotation_type': 386,
  'annotationversion_set': [],
  'blurred': None,
  

## Check that no error occurred and afterwards upload annotations

In [13]:
#annotations_api.create_annotation(body=transformed_annotations_to_upload)

[{'annotation_type': 384,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2755406,
  'image': 10410,
  'last_edit_time': datetime.datetime(2021, 1, 10, 17, 28, 56, 770035),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 17, 28, 56, 770002),
  'unique_identifier': '95183f6e-0a00-4de4-9c97-fedfec655a27',
  'uploaded_media_files': [],
  'user': 1,
  'vector': {'x1': 20328, 'x2': 20382, 'y1': 16747, 'y2': 16801},
  'verified_by_user': 'False'},
 {'annotation_type': 385,
  'annotationversion_set': [],
  'blurred': 'False',
  'concealed': 'False',
  'deleted': False,
  'description': '',
  'id': 2755407,
  'image': 10410,
  'last_edit_time': datetime.datetime(2021, 1, 10, 17, 28, 56, 770124),
  'last_editor': 1,
  'meta_data': None,
  'time': datetime.datetime(2021, 1, 10, 17, 28, 56, 770094),
  'unique_identifier': 'cc3def2b-5631-4619-ab08-51f2a69c667d',
  'uploaded_media_files':